<a href="https://colab.research.google.com/github/ludanortmun/itesm-mna-barred-galaxies/blob/new-models/notebooks/ModelosCNNAlternativos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos alternativos - CNNs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/MyDrive/datasets/bargal/dataset.processed.GRLogDiff.zip' -d './data/processed'

Streaming output truncated to the last 5000 lines.
  inflating: ./data/processed/manga-8319-6102.png  
  inflating: ./data/processed/manga-8319-6103.png  
  inflating: ./data/processed/manga-8319-6104.png  
  inflating: ./data/processed/manga-8319-9101.png  
  inflating: ./data/processed/manga-8319-9102.png  
  inflating: ./data/processed/manga-8320-12701.png  
  inflating: ./data/processed/manga-8320-12702.png  
  inflating: ./data/processed/manga-8320-12703.png  
  inflating: ./data/processed/manga-8320-12704.png  
  inflating: ./data/processed/manga-8320-12705.png  
  inflating: ./data/processed/manga-8320-1901.png  
  inflating: ./data/processed/manga-8320-1902.png  
  inflating: ./data/processed/manga-8320-3701.png  
  inflating: ./data/processed/manga-8320-3702.png  
  inflating: ./data/processed/manga-8320-3703.png  
  inflating: ./data/processed/manga-8320-3704.png  
  inflating: ./data/processed/manga-8320-6101.png  
  inflating: ./data/processed/manga-8320-6102.png  
  inflat

In [3]:
# Uncomment this if running in Google Colab. It will install the bargal package from GitHub.
!pip install git+https://github.com/ludanortmun/itesm-mna-barred-galaxies.git

  Cloning https://github.com/ludanortmun/itesm-mna-barred-galaxies.git to /tmp/pip-req-build-lcbgadk7
  Running command git clone --filter=blob:none --quiet https://github.com/ludanortmun/itesm-mna-barred-galaxies.git /tmp/pip-req-build-lcbgadk7
  Resolved https://github.com/ludanortmun/itesm-mna-barred-galaxies.git to commit 89df3ba4931fbd0abd508eed04cf3de4f2f0a423
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.4/863.4 kB 59.3 MB/s eta 0:00:00
  Created wheel for bargal: filename=bargal-0.1.0-py3-none-any.whl size=80759 sha256=a0b353f884b8df2b020d5c08fd7c1532e64694cddf8be9168a4f606026c90fcc
  Stored in directory: /tmp/pip-ephem-wh

In [1]:
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score, accuracy_score, \
    precision_score
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from PIL import Image

from bargal.dataset.load import load_dataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


# Preparando el conjunto de datos

El primer paso consiste en cargar nuestro conjunto de datos y dividirlo en conjunto de entrenamiento, validación y prueba.

In [3]:
dataset_path = './data/dataset.csv'

df = load_dataset(dataset_path)

train_df, val_test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['Bars'])
valid_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42, stratify=val_test_df['Bars'])

print(f'Train size: {len(train_df)}')
print(f'Validation size: {len(valid_df)}')
print(f'Test size: {len(test_df)}')

Train size: 7088
Validation size: 1519
Test size: 1519


Ahora creamos nuestra clase para representar el conjunto de datos, heredando de Dataset de PyTorch. El dataset a utilizar para nuestra red neuronal consiste en imágenes de galaxias y su respectiva etiqueta, que indica si la galaxia tiene o no una barra.

La etiqueta es un tensor de tamaño 1, donde 1 indica que la galaxia tiene una barra y 0 indica que no. Sin embargo, el conjunto de datos original no tiene etiquetas binarias, sino múltiples categorías indicando el tipo de barra que tiene la galaxia. Por lo tanto, convertimos cualquier etiqueta que represente la presencia de una barra (independientemente de sus características) a 1 y cualquier etiqueta que represente la ausencia de una barra a 0. También debemos filtrar elementos con la etiqueta -0.5, la cual simboliza que la categoria de la galaxia es desconocida. Estas transformaciones fueron exploradas previamente en el entregable [Avance1.Equipo22.ipynb](https://github.com/ludanortmun/itesm-mna-barred-galaxies/tree/main/notebooks/Avance1.Equipo22.ipynb), donde se creó la columna `has_bar` derivada de `Bars` y se filtraron los elementos sin clasificación.

En cuanto a la carga de imágenes, este conjunto de datos consiste en las imágenes de galaxias sin procesamiento, en formato JPG a color. El preprocesamiento consiste, principalmente, en la sustracción de las bandas G y R para enfatizar las estructuras de barras. Las imágenes resultantes tienen dimensiones de 400x400 píxeles y están en escala de grises. Sin embargo, debido a que esta red neuronal espera imágenes con 3 canales de entrada, la imagen es convertida a representación RGB. Estas imágenes son cargadas utilizando la librería PIL y convertidas a tensores.

El script de preprocesamiento puede ser consultado en este enlace: [bargal/commands/preprocess.py](https://github.com/ludanortmun/itesm-mna-barred-galaxies/blob/297f69b278ea6bc5099ef23a0d539602995bc55e/bargal/commands/preprocess.py)

El conjunto de imágenes pre procesadas puede descargarse con el siguiente enlace: [dataset.processed.GRLogDiff](https://tecmx-my.sharepoint.com/:u:/g/personal/a01795197_tec_mx/EexaLnqaLLdCt1JNxLib8VYBeOHJo95vuOr-Pfxv-55Iww?e=0gfeuq)


In [4]:
class GalaxiesDataset(Dataset):
    def __init__(self, galaxies_df: pd.DataFrame, img_dir: str):
        self.filtered_df = galaxies_df[galaxies_df['Bars'] >= 0].reset_index(drop=True)
        self.img_dir = img_dir

    def __len__(self):
        return len(self.filtered_df)

    def __getitem__(self, idx):
        row = self.filtered_df.iloc[idx]
        label = 1.0 if row['Bars'] != 0 else 0.0
        img_path = f"{self.img_dir}/{row['name']}.png"

        # Load image and convert to tensor
        with Image.open(img_path) as img:
            # Convert grayscale to RGB
            image = transforms.ToTensor()(img.convert('RGB'))

        return image.to(device), torch.tensor([label], dtype=torch.float32).to(device)

In [5]:
n = 32
processed_images_path = './data/processed'

train_data = GalaxiesDataset(train_df, processed_images_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_data = GalaxiesDataset(valid_df, processed_images_path)
valid_loader = DataLoader(valid_data, batch_size=n)
valid_N = len(valid_loader.dataset)

test_data = GalaxiesDataset(test_df, processed_images_path)
test_loader = DataLoader(test_data, batch_size=n)
test_N = len(test_loader.dataset)

# Modelo 1: MobileNetV3

## Definiendo el modelo

### Modelo pre-entrenado base

Para este modelo, se utilizará la arquitectura MobileNet V3 en su variante Large pre-entrenada con el conjunto de datos de ImageNet.

In [6]:
from torchvision.models.mobilenet import mobilenet_v3_large
from torchvision.models import MobileNet_V3_Large_Weights

weights = MobileNet_V3_Large_Weights.DEFAULT
mobilenet_base_model = mobilenet_v3_large(weights=weights)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 94.7MB/s]


In [7]:
mobilenet_base_model.to(device)
mobilenet_base_model.requires_grad_(False)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

### Adaptando el modelo

Agregamos una capa lineal al final de la red neuronal. Esta capa sólo tiene 1 neurona de salida, puesto que estamos trabajando con un problema de clasificación binaria.

In [8]:
model_mobilenet = nn.Sequential(
    mobilenet_base_model,
    nn.Linear(1000, 1)
)

### Preparando función de pérdida y optimizador

Debido a que estamos trabajando con un problema de clasificación binaria, utilizaremos la función de pérdida BCEWithLogitsLoss. El optimizador a utilizar es Adam.

Debido a que nuestro problema es no balanceado y además nos interesa optimizar la métrica de recall, podemos configurar el argumento `pos_weight` de la función de pérdida para darle mayor peso a la clase positiva. Este argumento lo calculamos diviendo la cantidad de elementos de la clase negativa entre la cantidad de elementos de la clase positiva, lo cual deberá resultar en un valor mayor a 1.

In [14]:
pos_weight = train_data.filtered_df['Bars'][train_data.filtered_df['Bars'] == 0].count() / train_data.filtered_df['Bars'][train_data.filtered_df['Bars'] != 0].count()
print(f'Using pos_weight={pos_weight}')

Using pos_weight=1.6551984877126653


In [9]:
mobilenet_loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))
mobilenet_optimizer = Adam(model_mobilenet.parameters())
model_mobilenet = model_mobilenet.to(device)

## Entrenamiento

### Definiendo funciones de entrenamiento

In [10]:
def get_classification_metrics(y_pred, y_true):
    return {
        'accuracy': accuracy_score(y_true=y_true, y_pred=y_pred),
        'recall': recall_score(y_true=y_true, y_pred=y_pred),
        'f1': f1_score(y_true=y_true, y_pred=y_pred),
        'precision': precision_score(y_true=y_true, y_pred=y_pred),
    }

In [11]:
def train(model, optimizer, loss_fn) -> dict[str, float]:
    loss = 0
    y_pred = []
    y_train = []

    model.train()
    for x, y in train_loader:
        output = model(x)
        mobilenet_optimizer.zero_grad()
        batch_loss = loss_fn(output, y)
        batch_loss.backward()
        optimizer.step()

        y_train += [int(y[i]) for i in range(len(y))]
        predictions = torch.gt(output, torch.tensor([0]).to(device))
        y_pred += [int(predictions[i]) for i in range(len(y))]

        loss += batch_loss.item()

    return {
        'loss': loss,
        **get_classification_metrics(y_pred, y_train),
    }

In [12]:
def validate(model, loss_fn) -> dict[str, float]:
    loss = 0
    y_pred = []
    y_val = []

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_fn(output, y).item()

            y_val += [int(y[i]) for i in range(len(y))]
            predictions = torch.gt(output, torch.tensor([0]).to(device))
            y_pred += [int(predictions[i]) for i in range(len(y))]

    return {
        'loss': loss,
        **get_classification_metrics(y_pred, y_val),
    }

### Ejecución del entrenamiento

In [13]:
epochs = 10
mobilenet_train_metrics = []
mobilenet_valid_metrics = []

for epoch in range(epochs):
    print(f"Epoch {epoch}")
    train_metrics = train(model_mobilenet, mobilenet_optimizer, mobilenet_loss_fn)
    print(f"Train accuracy: {train_metrics['accuracy']:.4f} | Train recall: {train_metrics['recall']:.4f} | "
          f"Train precision: {train_metrics['precision']:.4f} | Train F1: {train_metrics['f1']:.4f} | "
          f"Train loss: {train_metrics['loss']:.4f}")
    mobilenet_train_metrics.append(train_metrics)

    valid_metrics = validate(model_mobilenet, mobilenet_loss_fn)
    print(f"Valid accuracy: {valid_metrics['accuracy']:.4f} | Valid recall: {valid_metrics['recall']:.4f} | "
          f"Valid precision: {valid_metrics['precision']:.4f} | Valid F1: {valid_metrics['f1']:.4f} | "
          f"Valid loss: {valid_metrics['loss']:.4f}")
    mobilenet_valid_metrics.append(valid_metrics)

Epoch 0
Train loss: 138.3744 | Train accuracy: 0.6476 | Train recall: 0.3376 | Train precision: 0.5526 | Train F1: 0.4192
Valid loss: 35.1351 | Valid accuracy: 0.4704 | Valid recall: 0.7919 | Valid precision: 0.3980 | Valid F1: 0.5298
Epoch 1
Train loss: 133.6992 | Train accuracy: 0.6634 | Train recall: 0.3909 | Train precision: 0.5786 | Train F1: 0.4666
Valid loss: 30.8628 | Valid accuracy: 0.6306 | Valid recall: 0.2081 | Valid precision: 0.5244 | Valid F1: 0.2980
Epoch 2
Train loss: 131.7774 | Train accuracy: 0.6756 | Train recall: 0.4178 | Train precision: 0.5996 | Train F1: 0.4924
Valid loss: 29.6877 | Valid accuracy: 0.6638 | Valid recall: 0.2028 | Valid precision: 0.6805 | Valid F1: 0.3125
Epoch 3
Train loss: 131.2746 | Train accuracy: 0.6776 | Train recall: 0.4234 | Train precision: 0.6025 | Train F1: 0.4973
Valid loss: 29.6827 | Valid accuracy: 0.6545 | Valid recall: 0.5379 | Valid precision: 0.5417 | Valid F1: 0.5398
Epoch 4
Train loss: 130.0992 | Train accuracy: 0.6828 | Trai

KeyboardInterrupt: 

## Fine-Tunning

El entrenamiento realizado sólamente actualizó la última capa agregada para realizar la clasificación binaria. Sin embargo, podemos realizar fine-tunning para mejorar aún más el rendimiento de nuestro modelo. Para esto, "descongelamos" el modelo base y repetimos el entrenamiento. Nótese que se ejecutaron sólo 2 épocas y se utilizó un learning rate bastante pequeño.

In [ ]:
mobilenet_base_model.requires_grad_(True)
mobilenet_optimizer = Adam(model_mobilenet.parameters(), lr=0.000001)

In [ ]:
epochs = 2

for epoch in range(epochs):
    print(f"Epoch {epoch}")
    train_metrics = train(model_mobilenet, mobilenet_optimizer, mobilenet_loss_fn)
    print(f"Train accuracy: {train_metrics['accuracy']:.4f} | Train recall: {train_metrics['recall']:.4f} | "
          f"Train precision: {train_metrics['precision']:.4f} | Train F1: {train_metrics['f1']:.4f} | "
          f"Train loss: {train_metrics['loss']:.4f}")
    mobilenet_train_metrics.append(train_metrics)

    valid_metrics = validate(model_mobilenet, mobilenet_loss_fn)
    print(f"Valid accuracy: {valid_metrics['accuracy']:.4f} | Valid recall: {valid_metrics['recall']:.4f} | "
          f"Valid precision: {valid_metrics['precision']:.4f} | Valid F1: {valid_metrics['f1']:.4f} | "
          f"Valid loss: {valid_metrics['loss']:.4f}")
    mobilenet_valid_metrics.append(valid_metrics)

## Evaluación


In [ ]:
model_mobilenet.requires_grad_(False)
model_mobilenet.eval()

In [ ]:
y_test = []
y_pred = []

for x, y in test_loader:
    y_test += [int(y[i]) for i in range(len(y))]

    output = model_mobilenet(x)
    predictions = torch.gt(output, torch.tensor([0]).to(device))
    y_pred += [int(predictions[i]) for i in range(len(y))]

In [ ]:
mobilenet_test_metrics = get_classification_metrics(y_pred, y_test)

print(f"Train accuracy: {mobilenet_test_metrics['accuracy']:.4f}")
print(f"Train recall: {mobilenet_test_metrics['recall']:.4f}")
print(f"Train precision: {mobilenet_test_metrics['precision']:.4f}")
print(f"Train F1: {mobilenet_test_metrics['f1']:.4f}")

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Barrada", "Barrada"], yticklabels=["No Barrada", "Barrada"])
plt.title("Matriz de Confusión")
plt.xlabel("Predicción")
plt.ylabel("Valor Real")
plt.show()

In [ ]:
print("Reporte de Clasificación")
print(classification_report(y_test, y_pred, target_names=["No Barrada", "Barrada"]))

In [ ]:
def plot_metrics_comparison(train_metrics, valid_metrics, test_metrics):
    metrics = ['accuracy', 'recall', 'precision', 'f1']
    sets = ['Train', 'Validation', 'Test']

    # Set up the figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('Model Performance Metrics Comparison')

    # Get metrics values
    values = {
        'Train': train_metrics,
        'Validation': valid_metrics,
        'Test': test_metrics
    }

    # Plot each metric
    for idx, metric in enumerate(metrics):
        ax = axes[idx//2, idx%2]
        x = np.arange(len(sets))
        metric_values = [values[s][metric] for s in sets]

        ax.bar(x, metric_values)
        ax.set_title(metric.capitalize())
        ax.set_xticks(x)
        ax.set_xticklabels(sets)
        ax.set_ylim(0, 1)
        ax.grid(True, alpha=0.3)

        # Add value labels on top of bars
        for i, v in enumerate(metric_values):
            ax.text(i, v, f'{v:.3f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

In [ ]:
plot_metrics_comparison(mobilenet_train_metrics[-1], mobilenet_valid_metrics[-1], mobilenet_test_metrics)

## Guardando el modelo

In [ ]:
model_path = '/content/drive/MyDrive/datasets/bargal/model_mobile_netv3.pth'
torch.save(model_mobilenet.state_dict(), model_path)

# Modelo 2: AlexNet

## Definiendo el modelo

### Modelo pre-entrenado base

Para este modelo, se utilizará la arquitectura AlexNet pre-entrenada con el conjunto de datos de ImageNet.

In [ ]:
from torchvision.models.alexnet import alexnet
from torchvision.models import AlexNet_Weights

weights = AlexNet_Weights.DEFAULT
alexnet_base_model = alexnet(weights=weights)

In [ ]:
alexnet_base_model.to(device)
alexnet_base_model.requires_grad_(False)

### Adaptando el modelo

Agregamos una capa lineal al final de la red neuronal. Esta capa sólo tiene 1 neurona de salida, puesto que estamos trabajando con un problema de clasificación binaria.

In [ ]:
model_alexnet = nn.Sequential(
    alexnet_base_model,
    nn.Linear(1000, 1)
)

### Preparando función de pérdida y optimizador

Debido a que estamos trabajando con un problema de clasificación binaria, utilizaremos la función de pérdida BCEWithLogitsLoss. El optimizador a utilizar es Adam. Nuevamente utilizamos el valor de pos_weight calculado preiamente.

In [ ]:
alexnet_loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))
alexnet_optimizer = Adam(model_alexnet.parameters())
model_alexnet = model_alexnet.to(device)

## Entrenamiento

No es necesario volver a definir nuestras funciones de entrenamiento, por lo que podemos pasar directamente a su ejecución.

In [ ]:
epochs = 10
alexnet_train_metrics = []
alexnet_valid_metrics = []

for epoch in range(epochs):
    print(f"Epoch {epoch}")
    train_metrics = train(model_alexnet, alexnet_optimizer, alexnet_loss_fn)
    print(f"Train accuracy: {train_metrics['accuracy']:.4f} | Train recall: {train_metrics['recall']:.4f} | "
          f"Train precision: {train_metrics['precision']:.4f} | Train F1: {train_metrics['f1']:.4f} | "
          f"Train loss: {train_metrics['loss']:.4f}")
    alexnet_train_metrics.append(train_metrics)

    valid_metrics = validate(model_alexnet, alexnet_loss_fn)
    print(f"Valid accuracy: {valid_metrics['accuracy']:.4f} | Valid recall: {valid_metrics['recall']:.4f} | "
          f"Valid precision: {valid_metrics['precision']:.4f} | Valid F1: {valid_metrics['f1']:.4f} | "
          f"Valid loss: {valid_metrics['loss']:.4f}")
    alexnet_valid_metrics.append(valid_metrics)

## Fine-Tunning

El entrenamiento realizado sólamente actualizó la última capa agregada para realizar la clasificación binaria. Sin embargo, podemos realizar fine-tunning para mejorar aún más el rendimiento de nuestro modelo. Para esto, "descongelamos" el modelo base y repetimos el entrenamiento. Nótese que se ejecutaron sólo 2 épocas y se utilizó un learning rate bastante pequeño.

In [ ]:
alexnet_base_model.requires_grad_(True)
alexnet_optimizer = Adam(model_alexnet.parameters(), lr=0.000001)

In [ ]:
epochs = 2

for epoch in range(epochs):
    print(f"Epoch {epoch}")
    train_metrics = train(model_alexnet, alexnet_optimizer, alexnet_loss_fn)
    print(f"Train accuracy: {train_metrics['accuracy']:.4f} | Train recall: {train_metrics['recall']:.4f} | "
          f"Train precision: {train_metrics['precision']:.4f} | Train F1: {train_metrics['f1']:.4f} | "
          f"Train loss: {train_metrics['loss']:.4f}")
    alexnet_train_metrics.append(train_metrics)

    valid_metrics = validate(model_alexnet, alexnet_loss_fn)
    print(f"Valid accuracy: {valid_metrics['accuracy']:.4f} | Valid recall: {valid_metrics['recall']:.4f} | "
          f"Valid precision: {valid_metrics['precision']:.4f} | Valid F1: {valid_metrics['f1']:.4f} | "
          f"Valid loss: {valid_metrics['loss']:.4f}")
    alexnet_valid_metrics.append(valid_metrics)

## Evaluación


In [ ]:
model_alexnet.requires_grad_(False)
model_alexnet.eval()

In [ ]:
y_test = []
y_pred = []

for x, y in test_loader:
    y_test += [int(y[i]) for i in range(len(y))]

    output = model_alexnet(x)
    predictions = torch.gt(output, torch.tensor([0]).to(device))
    y_pred += [int(predictions[i]) for i in range(len(y))]

In [ ]:
alexnet_test_metrics = get_classification_metrics(y_pred, y_test)

print(f"Train accuracy: {alexnet_test_metrics['accuracy']:.4f}")
print(f"Train recall: {alexnet_test_metrics['recall']:.4f}")
print(f"Train precision: {alexnet_test_metrics['precision']:.4f}")
print(f"Train F1: {alexnet_test_metrics['f1']:.4f}")

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["No Barrada", "Barrada"], yticklabels=["No Barrada", "Barrada"])
plt.title("Matriz de Confusión")
plt.xlabel("Predicción")
plt.ylabel("Valor Real")
plt.show()

In [ ]:
print("Reporte de Clasificación")
print(classification_report(y_test, y_pred, target_names=["No Barrada", "Barrada"]))

In [ ]:
plot_metrics_comparison(alexnet_train_metrics[-1], alexnet_valid_metrics[-1], alexnet_test_metrics)

## Guardando el modelo

In [ ]:
model_path = '/content/drive/MyDrive/datasets/bargal/model_alexnet.pth'
torch.save(model_mobilenet.state_dict(), model_path)